<h1><center>University of Edinburgh</center></h1>
<h1><center>Geomagnetism (EASC10036)</center></h1>
<h1><center>Magnetic Field Mapping: From Measurements to Models</center></h1>

# (1.) Observations of the field

The magnetic field is measured in various ways - to produce modern global field models we typically use a combination of satellite and ground observatory measurements. In each case we make vector measurements to fully describe the field.

## Question 1

The observatory at Tristan da Cuhna is located in the South Atlantic, approximately midway between Africa and South America. The following field observations were made at the observatory:

| Component | Value |
| :- | -: |
| Horizontal intensity (H) | 10,301 nT |
| Declination angle (D) | -22.7&deg; |
| Inclination angle (I) | -64.4&deg; |

Sketch the relationship between these components in their orientation at Tristan Da Cuhna, and the remaining four standard field components. The remaining components are vertical intensity (Z), total intensity (F), North intensity (X), and East intensity (Y).

Calculate the total intensity of the field at Tristan da Cuhna, and comment on why this value is unusual.

**[6%]**

In [ ]:
### Import notebook dependencies, run these command first to set up the notebook

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

sys.path.append('..')
from src import sha_lib as sha, mag_lib as mag

# (2.) Plotting associated Legendre polynomials $P_n^m$
The $P_n^m(\theta)$ are building blocks for computing geomagnetic field models given a spherical harmonic model. It's instructive to visualise these functions and below you can experiment by setting different values of spherical harmonic degree ($n$) and order ($m \le n$). Note how the choice of $n$ and $m$ affects the number of zeroes of the functions. 

The functions are plotted on a semi-circle representing the surface of the Earth, with the inner core added for cosmetic purposes only! Again, purely for cosmetic purposes, the functions are scaled to fit within $\pm$20% of the Earth's surface.

### >> Exercise 1: USER INPUT HERE: Set the spherical harmonic degree and order for the plot

### 1D Associated Legrendre Polynomials

Modify the degree and order to examine the variation of the Associated Legrendre Polynomials ($P_n^m$) along a 1D meridional slice of the Earth (the core in grey is shown for visualisation). Try value such as degree $n=13$, order $m=0$ and others.

In [ ]:
degree = 5
order  = 3

In [ ]:
# Calculate Pnm and Xmn values every 0.5 degrees
colat   = np.linspace(0,180,361)
pnmvals = np.zeros(len(colat))
xnmvals = np.zeros(len(colat))

idx     = sha.pnmindex(degree,order)
for i, cl in enumerate(colat):
    p,x = sha.pxyznm_calc(degree, cl)[0:2]
    pnmvals[i] = p[idx]
    xnmvals[i] = x[idx]
    
theta   = np.deg2rad(colat)
ct      = np.cos(theta)
st      = np.sin(theta)

# Numbers mimicking the Earth's surface and outer core radii
e_rad   = 6.371
c_rad   = 3.485

# Scale values to fit within 10% of "Earth's surface". Firstly the P(n,m),
shell   = 0.2*e_rad
pmax    = np.abs(pnmvals).max()
pnmvals = pnmvals*shell/pmax + e_rad
xp      = pnmvals*st
yp      = pnmvals*ct


# Values to draw the Earth's and outer core surfaces as semi-circles
e_xvals = e_rad*st
e_yvals = e_rad*ct
c_xvals = e_xvals*c_rad/e_rad
c_yvals = e_yvals*c_rad/e_rad

# Earth-like background framework for plots
def eplot(ax):
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.plot(e_xvals,e_yvals, color='blue')
    ax.plot(c_xvals,c_yvals, color='black')
    ax.fill_between(c_xvals, c_yvals, y2=0, color='lightgrey')
    ax.plot((0, 0), (-e_rad, e_rad), color='black')

# Plot the P(n,m) 
fig, axes = plt.subplots(figsize=(18, 8))
#fig.suptitle('Degree (n) = '+str(degree)+', order (m) = '+str(order), fontsize=20)
                    
axes.plot(xp,yp, color='red')
axes.set_title('P('+ str(degree)+',' + str(order)+')', fontsize=16)
eplot(axes)


# (3.) Plotting 2D representations of the $P_n^m$



## Calculating geomagnetic field values 
The function below calculates geomagnetic field values at a point defined by its colatitude, longitude and altitude, using a spherical harmonic model of maximum degree _nmax_ supplied as an array _gh_. The parameter _coord_ is a string specifying whether the input position is in geocentric coordinates (when _altitude_ should be the geocentric distance in km) or geodetic coordinates (when altitude is distance above mean sea level in km). 

Though it's unconventional, we've chosen to include a monopole term, set to zero, at index zero in the _gh_ array.<br>

In [ ]:
def shm_calculator(gh, nmax, altitude, colat, long, coord):
    RREF     = 6371.2 #The reference radius assumed by the IGRF
    degree   = nmax
    phi      = long

    if (coord == 'Geodetic'):
        # Geodetic to geocentric conversion using the WGS84 spheroid
        rad, theta, sd, cd = sha.gd2gc(altitude, colat)
    else:
        rad   = altitude
        theta = colat

    # Function 'rad_powers' to create an array with values of (a/r)^(n+2) for n = 0,1, 2 ..., degree
    rpow = sha.rad_powers(degree, RREF, rad)

    # Function 'csmphi' to create arrays with cos(m*phi), sin(m*phi) for m = 0, 1, 2 ..., degree
    cmphi, smphi = sha.csmphi(degree,phi)

    # Function 'gh_phi_rad' to create arrays with terms such as [g(3,2)*cos(2*phi) + h(3,2)*sin(2*phi)]*(a/r)**5 
    ghxz, ghy = sha.gh_phi_rad(gh, degree, cmphi, smphi, rpow)

    # Function 'pnm_calc' to calculate arrays of the Associated Legendre Polynomials for n (&m) = 0,1, 2 ..., degree
    pnm, xnm, ynm, znm = sha.pxyznm_calc(degree, theta)

    # Geomagnetic field components are calculated as a dot product
    X =  np.dot(ghxz, xnm)
    Y =  np.dot(ghy,  ynm)
    Z =  np.dot(ghxz, znm)

    # Convert back to geodetic (X, Y, Z) if required
    if (coord == 'Geodetic'):
        t = X
        X = X*cd + Z*sd
        Z = Z*cd - t*sd

    return((X, Y, Z))

Enter the geomagnetic element to plot below: <br>
D = declination <br>
H = horizontal intensity <br>
I = inclination <br>
X = north component <br>
Y = east component <br>
Z = vertically (downwards) component <br>
F = total intensity.

Insert some values for the first three Gauss coefficients in the variable *ghp*.

- Try [1, 0, 0] - boring!
- Try [1,1,1] - interesting.
- Look the different components (e.g. 'D', 'Z', 'F', 'I')

- Can you explain why [1,0,0] with 'D' generates a blank image?
- Try [1,0,0] with 'I' - why do the inclination and latitude contours not match? Why is the inclination angle reversed relative to the general convention seen in the real Earth (i.e. positive in the northern hemisphere)? How would you fix this? What does it imply about the Earth's magnetic field?


### >> Exercise 2: USER INPUT HERE: Set the input parameters

### Summing together Associated Legrendre Polynomials and spherical harmonics

By summing together the Associated Legendre Polynomial basis functions (functions of colatitude) with the spherical harmonic basis terms (functions of longitude) that are weighted by the Gauss coefficients, we can represent arbitrarily complex spatial patterns of the magnetic field. Recall that the spatial complexity depends on the degree and order. In this exercise, change the values of the numbers in the _ghp_ variable to plot out 2D maps of the field.

Try out variations of other components and varying the _ghp_ values for each coefficient.

In [ ]:
ghp = np.empty([3,1]) # Set up an empty array and fill it with the values for the first three Gauss coefficients
ghp[0] = 1  # in nT
ghp[1] = 1
ghp[2] = 0
el2plot = 'Z'  # element to plot e.g, 'F', 'Z', 'D'

In [ ]:
def field_plotter(el_name, vals):
    # Function to plot map of field values
    if el_name=='D':
        cvals = np.arange(-25,30,5)  # If using 'D', limit the contours to -25 to +30 degrees
    else:
        cvals = 15  # Use 15 intervals
    fig, ax = plt.subplots(figsize=(16, 8))
    cplt = ax.contourf(longs, lats, vals, levels=cvals)
    cbar = fig.colorbar(cplt)
    ax.set_xlabel('Longitude', fontsize=16)
    ax.set_ylabel('Latitude', fontsize=16)
    ax.set_title('Degree 1 only', fontsize=16)
    if el_name=='D' or el_name=='I':
        cbar.set_label(str(el_name) + ' (degrees)', rotation=270, fontsize=14)
    else:
        cbar.set_label(str(el_name) + ' (nT)', rotation=270, fontsize=14)

In [ ]:
# Make a regular grid of latitude, longitude points to calculate the field at
nlats = 40
nlons = 91
degree = 1
longs  = np.linspace(-180, 180, nlons)
lats   = np.linspace(-80, 80, nlats)
ghp = np.append(0., ghp)
Bx, By, Bz = zip(*[sha.shm_calculator(ghp,degree,6371.2,90-lat,lon,'Geocentric') \
                 for lat in lats for lon in longs])
X = np.asarray(Bx).reshape(nlats,nlons)
Y = np.asarray(By).reshape(nlats,nlons)
Z = np.asarray(Bz).reshape(nlats,nlons)
D, H, I, F = [mag.xyz2dhif(X, Y, Z)[el] for el in range(4)]

# Plot the desired field component map
el_dict={'X':X, 'Y':Y, 'Z':Z, 'D':D, 'H':H, 'I':I, 'F':F}
field_plotter(el2plot, el_dict[el2plot])

# (4.) The International Geomagnetic Reference Field
The latest version of the IGRF is IGRF13 which consists of a main-field model every five years from 1900.0 to 2020.0 and a secular variation model for 2020-2025. The main field models have spherical harmonic degree (n) and order (m) 10 up to 1995 and n=m=13 from 2000 onwards. The secular variation model has n=m=8.

The coefficients are first loaded into a pandas (pd) dataframe: 

In [ ]:
IGRF13_FILE = os.path.abspath('../external/IGRF13coeffs.txt')
igrf13 = pd.read_csv(IGRF13_FILE, delim_whitespace=True,  header=3)
igrf13.head(10)  # Check the values have loaded correctly

## Question 2a

The scalar potential of the geomagnetic field can be written as follows:

$$\begin{align}
\Omega(\theta,\phi,r) = \frac{a}{\mu_0} \sum_{n=1}^{\inf} \left[ \sum_{m=1}^n \left(\frac{a}{r}\right)^{n+1} \left( g_n^m\cos(m\phi) + h_n^m\sin(m\phi) \right) \\
+ \left( \frac{r}{a}\right)^n \left( q_n^m\cos(m\phi) + s_n^m\sin(m\phi) \right) \right] P_n^m(\cos(\theta))
\end{align}$$

Name and briefly explain each of the terms in this equation, and note the SI unit where applicable.

**[6%]**

## Question 2b

Describe with the aid of an equation how the magnetic potential $\Omega$ and magnetic induction $\bf{B}$ relate to each other.

What is the main assumption we must make for this relation to be valid?

**[3%]**

## Question 2c

According to the IGRF model, the dipole component of the field in 1900 can be described by the Gauss coefficients with the following values:

| Gauss coefficient | Value (nT) |
| :- | -: |
| $g_1^0$ | -31,543 |
|$g_1^1$ | -2298 |
| $h_1^1$ | 5922 |

Edinburgh is located at approximately 56&deg;N, 0&deg;E, and is at sea level (0km altitude), which is locally equivalent to a spherical radius of 6363.5km.

Based on the equations from (2a) and (2b), derive the equations for the vector components of the dipole magnetic field in the X, Y, and Z directions, for internal field sources only.

Hence, calculate the field components values F, H, D and I.

Note that $P_1^0(\cos\theta)=\cos\theta$, and $P_1^1(\cos\theta)=\sin\theta$, and Earth's mean spherical radius is 6,371.2km.


**[10%]**

# Question 2d

Using the date and position given for Edinburgh in (2c), set the inputs below to calculate field values to degree $n=13$.

Run the next cell to compute the X, Y, Z, H, D, I and F values for the field. What values do you get, and why do they differ from your manually computed values from (2c)?

Using the online BGS IGRF calculator at
https://geomag.bgs.ac.uk/data_service/models_compass/igrf_calc.html, you can input the given position and date for Edinburgh to compute values for both the field and its secular variation (SV).

How does the magnitude of the secular variation of F compare to the difference between your computed field values from (2c) and the first part of this question?

The airport runway at Edinburgh is painted with the declination value rounded to the nearest 10&deg;. Given the declination and its SV given by the online calculator, and assuming SV remains constant, what year would the runway next need to be repainted in order to remain accurate to the nearest 10&deg;?

**[8%]**

### >>  >> USER INPUT HERE: Set the input parameters
### Compute magnetic field values for a single location

In [ ]:
location = 'Place name'
ctype    = 'Geocentric'  # coordinate type
altitude = 6731.2          # in km above the spheroid if ctype = 'Geodetic', radial distance if ctype = 'Geocentric'
colat    = 42         # NB colatitude, not latitude
long     = 42         # longitude
date     = 2020.0      # Date for the field estimates
NMAX     = 10          # Maximum spherical harmonic degree of the model; Change this from 1 to 13 for the full IGRF model

Now calculate the IGRF geomagnetic field estimates.

In [ ]:
# Calculate the gh coefficient values for the supplied date
if date == 2020.0:
    gh = igrf13['2020.0']
elif date < 2020.0:
    date_1 = (date//5)*5
    date_2 = date_1 + 5
    w1 = date-date_1
    w2 = date_2-date
    gh = np.array((w2*igrf13[str(date_1)] + w1*igrf13[str(date_2)])/(w1+w2))
elif date > 2020.0:
    gh =np.array(igrf13['2020.0'] + (date-2020.0)*igrf13['2020-25'])

gh = np.append(0., gh) # Add a zero monopole term corresponding to g(0,0)

bxyz = shm_calculator(gh, NMAX, altitude, colat, long, ctype)
dec, hoz ,inc , eff = mag.xyz2dhif(bxyz[0], bxyz[1], bxyz[2])

# Print out the results
print('\nGeomagnetic field values at: ', location+', '+ str(date), '\n')
print('Declination (D):', '{: .2f}'.format(dec), 'degrees')
print('Inclination (I):', '{: .2f}'.format(inc), 'degrees')
print('Horizontal intensity (H):', '{: .1f}'.format(hoz), 'nT')
print('Total intensity (F)     :', '{: .1f}'.format(eff), 'nT')
print('North component (X)     :', '{: .1f}'.format(bxyz[0]), 'nT')
print('East component (Y)      :', '{: .1f}'.format(bxyz[1]), 'nT')
print('Vertical component (Z)  :', '{: .1f}'.format(bxyz[2]), 'nT')

# (5.) Maps of the IGRF
Now draw maps of the IGRF at the date selected above. The latitude range is set at -80 degrees to +80 degrees and the longitude range -180 degrees to +180 degrees and IGRF values for (X, Y, Z) are calculated on a 5 degree grid (this may take a few seconds to complete).

## >>  >> USER INPUT HERE:  Set the element to plot

In [ ]:
ctype    = 'Geocentric'  # coordinate type
altitude = 6371.2          # in km above the spheroid if ctype = 'Geodetic', radial distance if ctype = 'Geocentric'
date     = 2020.0      # Date for the field estimates
NMAX     = 10          # Maximum spherical harmonic degree of the model; 
el2plot = 'X'

In [ ]:
# Calculate the gh coefficient values for the supplied date
if date == 2020.0:
    gh = igrf13['2020.0']
elif date < 2020.0:
    date_1 = (date//5)*5
    date_2 = date_1 + 5
    w1 = date-date_1
    w2 = date_2-date
    gh = np.array((w2*igrf13[str(date_1)] + w1*igrf13[str(date_2)])/(w1+w2))
elif date > 2020.0:
    gh =np.array(igrf13['2020.0'] + (date-2020.0)*igrf13['2020-25'])

gh = np.append(0., gh) # Add a zero monopole term corresponding to g(0,0)

In [ ]:
def IGRF_plotter(el_name, vals, date):
    # A function to plot a field component map for the IGRF
    if el_name=='D':
        cvals = np.arange(-30,30,2)
    else:
        cvals = 30
    fig, ax = plt.subplots(figsize=(16, 8))
    cplt = ax.contourf(longs, lats, vals, levels=cvals,cmap = 'seismic')
    #ax.clabel(cplt, cplt.levels, inline=True, fmt='%d', fontsize=10)
    fig.colorbar(cplt)
    ax.set_title('IGRF: '+ el_name + ' (' + str(date) + ')', fontsize=20)
    ax.set_xlabel('Longitude', fontsize=16)
    ax.set_ylabel('Latitude', fontsize=16)

In [ ]:
# Set up a latitude, longitude grid, and calculate the field at each grid point
nlats = 40
nlons = 91
longs  = np.linspace(-180, 180, nlons)
lats   = np.linspace(-80, 80, nlats)
Bx, By, Bz = zip(*[sha.shm_calculator(gh,NMAX,altitude,90-lat,lon,ctype) \
                 for lat in lats for lon in longs])
X = np.asarray(Bx).reshape(nlats,nlons)
Y = np.asarray(By).reshape(nlats,nlons)
Z = np.asarray(Bz).reshape(nlats,nlons)
D, H, I, F = [mag.xyz2dhif(X, Y, Z)[el] for el in range(4)]

# Plot the chosen field component map
el_dict={'X':X, 'Y':Y, 'Z':Z, 'D':D, 'H':H, 'I':I, 'F':F}
IGRF_plotter(el2plot, el_dict[el2plot], date)

## Question 3

Using the code cells above plot maps of the vertical field component at the core-mantle boundary (radius $c=3485km$), the Earth's surface (radius $a=6371.2km$), and at the distance of the Moon's orbit (radius $m=a+350,000km$).

At each radius, plot a map for both degree $n=1$ and degree $n=13$.

Describe the spatial features of the field at each radius and spatial scale, and explain if and/or why they differ. Explain why the maps for degree $n=1$ and degree $n=13$ do or do not differ at each radius.

Use the date of 1995.0. Use map images to illustrate your answer where relevant.

**[8%]**